# Chapter 5. 병행성과 병렬성

* **병행성(concurrency)**
    - 여러 일을 마치 동시에 하듯이 수행하는 것.
* **병렬성(parallelism)**
    - 실제로 여러 작업을 동시에 실행하는 것.
    - 작업 속도 향상 가능.
* 파이썬 사용시
    - 병행 프로그램을 쉽게 작성 가능.
    - 시스템 호출, 서브프로세스, C 확장을 이용한 병렬 작업도 가능.
    - 병행 파이썬 코드를 실제 병렬로 실행하게 만드는 것은 매우 어려움.

## Better Way 36. 자식 프로세스를 관리하려면 subprocess를 사용하자

* 파이썬의 관련 장점
    - 자식 프로세스 실행 및 관리용 라이브러리 존재.
        + 명령줄 유틸리티 등 다른 도구 연계에 유용.
    - 자식 프로세스를 병렬로 실행 가능.
        + CPU 코어를 모두 이용해 처리량 극대화 가능.
    - 파이썬 자체는 CPU 속도에 의존할 수 있지만(**BW 37** 참조) CPU를 많이 사용하는 작업 관리 및 조절이 용이.
    

* 파이썬에서 서브프로세스를 실행하는 방법
    - popen, popen2, os.exec* 등 존재.
    - **내장 모듈 subprocess 사용**
        + 최선이자 가장 간단한 방법.
  
  
* [예시] subprocess로 자식 프로세스 실행.
    - Popen 생성자가 프로세스 시작.
    - communicate 메서드가 자식 프로세스의 출력을 읽어오고 종료시까지 대기.

In [3]:
import subprocess

In [4]:
proc = subprocess.Popen(
    ['echo', 'Hello from the child!'],
    stdout=subprocess.PIPE)
out, err = proc.communicate()
print(out.decode('utf-8'))

Hello from the child!



* 자식 프로세스
    - 부모 프로세스 및 파이썬 인터프리터와 독립적으로 실행됨.
    - 자식 프로세스의 상태는 파이썬이 다른 작업을 하는 동안 주기적으로 폴링(polling)됨.

In [7]:
from time import sleep, time

proc = subprocess.Popen(['sleep', '0.3'])
while proc.poll() is None:
    print("Working...")
    # 시간이 걸리는 작업 몇 개를 수행함
    sleep(0.2)
print('Exit status', proc.poll())

Working...
Working...
Exit status 0


* 부모에서 자식 프로세스를 떼어낸다는 것은 부모 프로세스가 자유롭게 여러 자식 프로세스를 병렬로 실행할 수 있음을 의미.
    - 자식 프로세스를 떼어내려면 모든 자식 프로세스를 먼저 시작하면 됨.

In [9]:
def run_sleep(period):
    proc = subprocess.Popen(['sleep', str(period)])
    return proc

start = time()
procs = []
for _ in range(10):
    proc = run_sleep(0.1)
    procs.append(proc)

* 이후에는 communicate 메서드로 자식 프로세스들이 I/O를 마치고 종료하기를 기다리면 됨.

In [10]:
for proc in procs:
    proc.communicate()
end = time()
print('Finished in %.3f seconds' % (end - start))

Finished in 68.933 seconds


* **파이프(pipe)** 를 이용해 데이터를 서브프로세스로 보낸 다음 서브프로세스의 결과를 받아올 수도 있음.
    - 이를 이용해 다른 프로그램을 활용한 병렬 작업 수행 가능.


* [예시] 데이터 암호화에 openssl 명령줄 도구를 사용하는 경우.
    - 명령줄 인수와 I/O 파이프를 사용해 자식프로세스 실행.

In [16]:
import os

def run_openssl(data):
    env = os.environ.copy()
    env['password'] = '\xe24U\n\xd0Ql3S\x11' # b'\xe24U\n\xd0Ql3S\x11'
    proc = subprocess.Popen(
        ['openssl', 'enc', '-des3', '-pass', 'env:password'],
        env=env,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE)
    proc.stdin.write(data)
    proc.stdin.flush()  # Ensure the child gets input
    return proc

* 예제
    - 파이프로 암호화 함수에 임의의 바이트 전달.
* 실전
    - 사용자 입력, 파일 핸들, 네트워크 소켓 등 전달.

In [17]:
procs = []
for _ in range(3):
    data = os.urandom(10)
    proc = run_openssl(data)
    procs.append(proc)

* 자식 프로세스는 병렬로 실행되고 입력을 소비함.


* [예시] 자식 프로세스가 종료할 때까지 대기하고 최종 결과를 받는 코드.

In [18]:
for proc in procs:
    out, err = proc.communicate()
    print(out[-10:])

b'\x87\x14 \xf2/\x80\xb9BBA'
b'\xf0\xec\xe2x\x82Ux\xfbEg'
b'\x16;_\xb7 V\xf7\xb6\x9de'


* 유닉스의 파이프처럼 한 자식 프로세스의 결과를 다른 프로세스의 입력으로 연결하여 병렬 프로세스의 **체인(chain)** 을  생성할 수도 있음.


* [예시] 자식 프로세스를 시작하여 md5 명령줄 도구에서 입력 스트림을 소비하게 하는 함수.
    - 주의: 파이썬 내장 모듈 hashlib은 md5 함수를 제공하므로 subprocess를 항상 이렇게 실행할 필요는 없음.

In [28]:
def run_md5(input_stdin):
    proc = subprocess.Popen(
        ['md5sum'], # 운영체제마다 프로그램 이름이 다름
        stdin=input_stdin,
        stdout=subprocess.PIPE)
    return proc

* 이제 데이터를 암호화하는 openssl 프로세스 집합과 암호화된 결과를 md5로 해시(hash)하는 프로세스 집합을 시작 가능.

In [29]:
input_procs = []
hash_procs = []
for _ in range(3):
    data = os.urandom(10)
    proc = run_openssl(data)
    input_procs.append(proc)
    hash_proc = run_md5(proc.stdout)
    hash_procs.append(hash_proc)

* 일단 자식 프로세스들이 시작하면 이들 사이의 I/O는 자동으로 발생.
    - 모든 작업이 끝나고 최종 결과물이 출력되기를 기다리기만 하면 됨.

In [22]:
for proc in input_procs:
    proc.communicate()
    
for proc in hash_procs:
    out, err = proc.communicate()
    print(out.strip())

* 자식 프로세스가 종료되지 않거나 입력 또는 출력 파이프에서 블록될 염려가 있는 경우
    - communicate 메서드에 timeout 파라미터를 넘겨야 함.
         + 이렇게 하면 자식 프로세스가 일정한 시간 내에 응답하지 않을 때 예외가 일어나 오작동하는 자식 프로세스를 종료 가능.

In [23]:
proc = run_sleep(10)
try:
    proc.communicate(timeout=0.1)
except subprocess.TimeoutExpired:
    proc.terminate()
    proc.wait()

print('Exit status', proc.poll())

Exit status 1


* 주의: timeout 파라미터는 파이썬 3.3 이후 버전에서 사용 가능.
    - 이전 버전에서 I/O 타임아웃을 강제하려면 내장 모듈 select를 proc.stdin, proc.stdout, proc.stderr에 사용해야 함.
    

### 핵심 정리
* 자식 프로세스를 실행하고 자식 프로세스의 입출력 스트림을 관리하려면 subprocess 모듈을 사용하자.
* 자식 프로세스는 파이썬 인터프리터에서 병렬로 실행되어 CPU 사용을 극대화하게 해준다.
* communicate에 timeout 파라미터를 사용하여 자식 프로세스들이 교착 상태(deadlock)에 빠지거나 멈추는 상황을 막자.